# 지도학습-TruncatedSVD
## 1. 준비작업
### 1. 패키지 참조

In [1]:
from hossam import *
import numpy as np

from pandas import pivot_table, DataFrame, merge
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

📦 아이티윌 이광호 강사가 제작한 라이브러리를 사용중입니다.
📚 자세한 사용 방법은 https://py.hossam.kr 을 참고하세요.
📧 Email: leekh4232@gmail.com
🎬 Youtube: https://www.youtube.com/@hossam-codingclub
📝 Blog: https://blog.hossam.kr/
🔖 Version: 0.5.3

✅ 시각화를 위한 한글 글꼴(NotoSansKR-Regular)이 자동 적용되었습니다.


### 2. 데이터셋 가져오기

In [2]:
origin = load_data('card_useage')
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print(origin.info())
origin.head()

카드 사용량에 대한 데이터 (출처: 서울빅데이터캠퍼스)
데이터셋 크기: (262, 5)
열 개수: 5
행 개수: 262
<class 'pandas.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   기준일자      262 non-null    datetime64[us]
 1   가맹점주소(구)  262 non-null    str           
 2   업종대분류     262 non-null    str           
 3   카드이용금액계   262 non-null    int64         
 4   카드이용건수계   262 non-null    int64         
dtypes: datetime64[us](1), int64(2), str(2)
memory usage: 10.4 KB
None


,기준일자,가맹점주소(구),업종대분류,카드이용금액계,카드이용건수계
0,2016-01-09,노원구,가정생활/서비스,20120,5
1,2016-01-09,서대문구,요식/유흥,324883,10
2,2016-01-31,은평구,요식/유흥,5149865,5
3,2016-02-06,서대문구,유통,237919,50
4,2016-02-08,서초구,요식/유흥,180074,5


### 3. 잠재요인 발굴하기
#### 1. 구 X 업종 매출 행렬 생성

In [3]:
pivot_df = pivot_table(
    origin,
    index = '가맹점주소(구)',
    columns = '업종대분류',
    values = '카드이용금액계',
    aggfunc = 'sum',
    fill_value = 0          # NaN 값을 0으로 채움
)

pivot_df

업종대분류,가전/가구,가정생활/서비스,교육/학원,미용,스포츠/문화/레저,여행/교통,요식/유흥,유통,음/식료품,의료,의류/잡화,자동차,전자상거래,주유
가맹점주소(구),,,,,,,,,,,,,,
강남구,662049,955700,0,0,91546,0,301850,8031854,0,169763,0,26156,588510,235303
강동구,0,0,125750,0,0,0,315884,0,0,0,0,0,0,45270
강북구,236410,0,0,0,0,0,0,0,55330,0,0,0,0,0
강서구,0,30180,0,0,272626,0,197679,0,35210,0,0,0,123235,0
관악구,0,0,0,0,482377,201200,158445,635792,0,0,0,0,0,0
광진구,0,0,0,0,0,0,57845,510042,0,0,0,0,0,45270
구로구,61366,25150,0,16599,0,0,174038,635893,0,0,0,130780,0,0
구리시,0,0,0,0,0,0,0,0,0,0,0,0,65390,0
금천구,0,0,0,0,0,75450,0,0,0,101012,0,0,150900,0


#### 2. 로그 변환 (왜도 완화) + 데이터 스케일링

In [4]:
pivot_log = np.log1p(pivot_df)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(pivot_log)


#### 3. 전체 차원에 대한 SVD 분석 수행

In [5]:
# 최대 차원 설정
max_components = len(pivot_df.columns)

svd = TruncatedSVD(n_components = max_components, random_state = 52)
svd.fit(scaled_data)

# 설명분산 비율
explained_var = svd.explained_variance_ratio_

# 누적 설명분산
cum_explained_var = np.cumsum(explained_var)

# 표 생성
variance_df = DataFrame({
    'Component': range(1, max_components + 1),
    'Explained_Variance_Ratio': explained_var,
    'Cumulative_Explained_Variance': cum_explained_var
})

variance_df

,Component,Explained_Variance_Ratio,Cumulative_Explained_Variance
0,1,0.239,0.239
1,2,0.132,0.371
2,3,0.123,0.495
3,4,0.098,0.592
4,5,0.080,0.673
5,6,0.075,0.747
6,7,0.066,0.813
7,8,0.054,0.867
8,9,0.045,0.913
9,10,0.033,0.946


#### 4. 분산 비율이 90%까지인 요인만으로 재학습

In [6]:
n_factors = 9
svd = TruncatedSVD(n_components = n_factors, random_state = 52)
area_factors = svd.fit_transform(scaled_data)
area_factors[:5]

array([[ 3.21104902, -1.18045744,  0.66791854,  0.46115067, -0.08633886,
        -1.24345233,  0.67685339,  0.2452793 ,  1.2717643 ],
       [-1.69682133, -0.29089156,  1.82098385, -1.43885638,  2.43420759,
         1.97049816,  1.87234129,  0.39648396,  0.39809684],
       [-2.80289456, -0.59232514,  0.12958908, -0.95933817, -1.31785775,
        -1.48584988, -0.24735401,  1.10068454,  0.57593458],
       [-0.37628832,  0.35467836, -1.83346807,  0.01369646,  0.75356266,
        -0.87358575,  0.95509618,  0.74235879, -1.15110041],
       [-1.00328009,  0.31591403, -0.8753253 ,  3.08446537,  0.33558007,
         1.26528356, -0.13078302, -0.46858438,  0.94557008]])

#### 5. 구별 잠재요인 점수

In [7]:
columns = []

for i in range(n_factors):
    columns.append(f'Factor_{i+1}')

columns

['Factor_1',
 'Factor_2',
 'Factor_3',
 'Factor_4',
 'Factor_5',
 'Factor_6',
 'Factor_7',
 'Factor_8',
 'Factor_9']

In [8]:
factor_df = DataFrame(
    area_factors,
    columns = columns,
    index = pivot_df.index
)
factor_df

,Factor_1,Factor_2,Factor_3,Factor_4,Factor_5,Factor_6,Factor_7,Factor_8,Factor_9
가맹점주소(구),,,,,,,,,
강남구,3.211,-1.180,0.668,0.461,-0.086,-1.243,0.677,0.245,1.272
강동구,-1.697,-0.291,1.821,-1.439,2.434,1.970,1.872,0.396,0.398
강북구,-2.803,-0.592,0.130,-0.959,-1.318,-1.486,-0.247,1.101,0.576
강서구,-0.376,0.355,-1.833,0.014,0.754,-0.874,0.955,0.742,-1.151
관악구,-1.003,0.316,-0.875,3.084,0.336,1.265,-0.131,-0.469,0.946
광진구,-0.923,0.543,0.761,0.099,1.169,0.144,-0.944,-0.606,1.165
구로구,0.734,0.249,2.279,1.326,-0.607,-0.901,-0.747,1.994,-0.470
구리시,-2.465,0.413,-0.432,-0.799,-1.426,0.093,0.787,0.342,0.306
금천구,-1.747,-0.504,-1.718,-0.140,-2.431,1.562,0.854,-0.310,1.112


#### 6. 업종별 요인 로딩 확인 (해석 핵심)

In [9]:
loading_df = DataFrame(
    svd.components_.T,
    columns = [f'Factor_{i+1}' for i in range(n_factors)],
    index = pivot_df.columns
)
loading_df

,Factor_1,Factor_2,Factor_3,Factor_4,Factor_5,Factor_6,Factor_7,Factor_8,Factor_9
업종대분류,,,,,,,,,
가전/가구,0.137,-0.285,0.444,-0.015,-0.341,-0.430,-0.018,0.054,0.240
가정생활/서비스,0.394,0.168,-0.158,-0.040,0.184,-0.215,0.084,0.473,0.001
교육/학원,-0.036,-0.218,0.382,-0.176,0.256,0.487,0.510,0.123,-0.139
미용,0.246,0.486,0.129,0.092,-0.289,-0.009,0.107,0.194,-0.232
스포츠/문화/레저,0.208,0.142,-0.295,0.441,0.137,-0.203,0.500,-0.026,0.086
여행/교통,-0.009,-0.210,-0.364,0.431,-0.180,0.508,-0.098,0.080,0.302
요식/유흥,0.343,-0.004,0.140,0.134,0.528,0.062,-0.202,-0.262,-0.371
유통,0.349,-0.025,0.253,0.402,-0.014,0.084,-0.376,-0.276,0.030
음/식료품,0.001,-0.347,-0.451,-0.229,0.108,-0.179,-0.307,0.184,-0.351


#### 인사이트: Factor별 상위 로딩 업종 기준 해석

- 요인 해석은 각 Facotr에서 절대값이 큰 로딩 업종을 중심으로 해석
- (+- 부호는 방향, 크기는 영향력 의미)
    - 요인 해석은 절대적 정답 없음
    - 부호는 상대 방향 의미

### 4. 예측 모형에 적용
#### 1. 기본 구조

In [10]:
df_model_1 = merge(origin, loading_df, left_on = '업종대분류', right_index = True)
df_model_1.head()

,기준일자,가맹점주소(구),업종대분류,카드이용금액계,카드이용건수계,Factor_1,Factor_2,Factor_3,Factor_4,Factor_5,Factor_6,Factor_7,Factor_8,Factor_9
0,2016-01-09,노원구,가정생활/서비스,20120,5,0.394,0.168,-0.158,-0.040,0.184,-0.215,0.084,0.473,0.001
1,2016-01-09,서대문구,요식/유흥,324883,10,0.343,-0.004,0.140,0.134,0.528,0.062,-0.202,-0.262,-0.371
2,2016-01-31,은평구,요식/유흥,5149865,5,0.343,-0.004,0.140,0.134,0.528,0.062,-0.202,-0.262,-0.371
3,2016-02-06,서대문구,유통,237919,50,0.349,-0.025,0.253,0.402,-0.014,0.084,-0.376,-0.276,0.030
4,2016-02-08,서초구,요식/유흥,180074,5,0.343,-0.004,0.140,0.134,0.528,0.062,-0.202,-0.262,-0.371


#### 2. 구 X 업종 적합도(벡터 내적값) 도출

In [11]:
Z = factor_df.values
Lambda = loading_df.iloc[:, n_factors].values

structure_matrix = Z @ Lambda.T

structure_df = DataFrame(
    structure_matrix,
    index = factor_df.index,
    columns = loading_df.index
)

df_model_2 = df_model_1.copy()

df_model_2['Structure_Fit'] = df_model_2.apply(
    lambda row: structure_df.loc[
        row['가맹점주소(구)'],
        row['업종대분류']
    ],
    axis = 1
)

df_model_2.head()

IndexError: single positional indexer is out-of-bounds

### 5. 구 단위 창업 추천 업종 도출
#### 1. 구 X 업종 적합도 행렬

In [ ]:
Z = factor_df.values
Lambda = loading_df.iloc[:, n_factors].values

structure_matrix = Z @ Lambda.T

structure_df = DataFrame(
    structure_matrix,
    index = factor_df.index,
    columns = loading_df.index
)

structure_df.head()

#### 2. 구별 상위 3개 업종 추천

In [ ]:
recommend_df = (
    structure_df.apply(lambda x: x.sort_values(ascending = False).head(3).index.tolist(), axis = 1)
)

recommend_df = recommend_df.to_frame(name = 'Top3_Industries')

recommend_df

In [ ]:
recommend_expanded = DataFrame(
    recommend_df['Top3_Industries'].tolist()
    index = recommend_df.index,
    columns = ['Rank1', 'Rank2','Rank3']
)

recommend_expanded